# Tarea 2

In [7]:
try:
    spark.stop()
except:
    pass


In [8]:
import os, sys
from pyspark.sql import SparkSession

os.environ["PYSPARK_PYTHON"] = sys.executable
os.environ["PYSPARK_DRIVER_PYTHON"] = sys.executable

spark = (
    SparkSession.builder
    .master("local[*]")
    .appName("NYC_TAXI_RDD")
    .config("spark.driver.bindAddress", "127.0.0.1")
    .config("spark.driver.host", "127.0.0.1")
    .config("spark.driver.extraJavaOptions", "-Djava.net.preferIPv4Stack=true")
    .config("spark.executor.extraJavaOptions", "-Djava.net.preferIPv4Stack=true")
    # para ver el error real si vuelve a crashear
    .config("spark.python.worker.faulthandler.enabled", "true")
    .config("spark.sql.execution.pyspark.udf.faulthandler.enabled", "true")
    # a veces ayuda en Windows
    .config("spark.python.worker.reuse", "false")
    .getOrCreate()
)

print("Spark listo ✅")
print("Python:", sys.executable)


Spark listo ✅
Python: C:\Users\Oscar Ferreira\venv_pyspark\Scripts\python.exe


In [9]:
path = r"C:\Users\Oscar Ferreira\OneDrive - AUTO LINEAS AMERICA SA DE CV\Escritorio\MCD\6 - DATOS MASIVOS\MCD-DATOS-MASIVOS-2025-OJF\yellow_tripdata_2025-01.parquet"
df = spark.read.parquet(path)
print("DF leído ✅")


DF leído ✅


In [13]:
print("Total columnas:", len(df.columns))
print("\nColumnas:\n")

for col in df.columns:
    print(col)


Total columnas: 20

Columnas:

VendorID
tpep_pickup_datetime
tpep_dropoff_datetime
passenger_count
trip_distance
RatecodeID
store_and_fwd_flag
PULocationID
DOLocationID
payment_type
fare_amount
extra
mta_tax
tip_amount
tolls_amount
improvement_surcharge
total_amount
congestion_surcharge
Airport_fee
cbd_congestion_fee


In [14]:
rdd = df.rdd
print("✅ RDD creado | Particiones:", rdd.getNumPartitions())
print("Ejemplo 1 fila:", rdd.take(1)[0])


✅ RDD creado | Particiones: 15
Ejemplo 1 fila: Row(VendorID=1, tpep_pickup_datetime=datetime.datetime(2025, 1, 1, 0, 18, 38), tpep_dropoff_datetime=datetime.datetime(2025, 1, 1, 0, 26, 59), passenger_count=1, trip_distance=1.6, RatecodeID=1, store_and_fwd_flag='N', PULocationID=229, DOLocationID=237, payment_type=1, fare_amount=10.0, extra=3.5, mta_tax=0.5, tip_amount=3.0, tolls_amount=0.0, improvement_surcharge=1.0, total_amount=18.0, congestion_surcharge=2.5, Airport_fee=0.0, cbd_congestion_fee=0.0)


In [15]:
total_viajes = rdd.count()
print("Total de viajes:", total_viajes)


Total de viajes: 3475226


In [16]:
dist_rdd = rdd.map(lambda x: x["trip_distance"]).filter(lambda v: v is not None)
total_rdd = rdd.map(lambda x: x["total_amount"]).filter(lambda v: v is not None)
tip_rdd   = rdd.map(lambda x: x["tip_amount"]).filter(lambda v: v is not None)

print("Trip Distance -> mean:", dist_rdd.mean(), "min:", dist_rdd.min(), "max:", dist_rdd.max())
print("Total Amount  -> mean:", total_rdd.mean(), "min:", total_rdd.min(), "max:", total_rdd.max())
print("Tip Amount    -> mean:", tip_rdd.mean(),   "min:", tip_rdd.min(),   "max:", tip_rdd.max())


Trip Distance -> mean: 5.855126178844117 min: 0.0 max: 276423.57
Total Amount  -> mean: 25.611291697287875 min: -901.0 max: 863380.37
Tip Amount    -> mean: 2.959812786276307 min: -86.0 max: 400.0
